In [16]:
# !pip install -r ../requirements.txt

In [17]:
import os
import re

import json
import numpy as np
from typing import Tuple


from langchain.llms import OpenAI

from langchain_experimental.tot.base import ToTChain
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity

##Load API key ------------------
f = open('../apikeys/api.txt', 'r')
key = f.readline()
os.environ['OPENAI_API_KEY'] = key

In [18]:
class MyChecker(ToTChecker):
    def evaluate(self,
        problem_description: str,
        thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        
        last_thought = thoughts[-1]
        clean_solution = last_thought.replace(" ", "").replace('"', "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return ThoughtValidity.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return ThoughtValidity.VALID_INTERMEDIATE
        else:
            return ThoughtValidity.INVALID


In [19]:
difficulty_map = {'base':2,
                    'easy': 2,
                    'medium': 2,
                    'hard': 3,
                    'expert':4}

In [22]:
# data_config = {}
datadir = '../src/data/'
models = ["text-davinci-003"]
for model in models:
    llm = OpenAI(temperature=0.1, max_tokens=512, model=model)
    data_config[model] = {}
    for file in os.listdir(datadir):

        tile_size = file.split('.')[0].split('_')[-1]
        grid = tile_size.split('x')[0]
        subgrid = str(int(np.sqrt(int(grid))))+'x'+str(int(np.sqrt(int(grid))))
        children_thoughts = difficulty_map[file.split('_')[0]]*int(grid)
        
        with open(datadir+file) as f:
            data = f.read()

        data = json.loads(data)

        sudoku_puzzle = data['sudoku_puzzle']
        sudoku_solution = data['sudoku_solution']

        problem_description = f"""
            {sudoku_puzzle}

            - This is a {tile_size} Sudoku puzzle.
            - The * represents a cell to be filled.
            - The | character separates rows.
            - At each step, replace one or more * with digits 1-{grid}.
            - There must be no duplicate digits in any row, column or {subgrid} subgrid.
            - Keep the known digits from previous valid thoughts in place.
            - Each thought can be a partial or the final solution.
            """.strip()


        temp_dict = {'grid_size':tile_size,
                    'children_thoughts':children_thoughts,
                    'problem_description': problem_description,
                    'initial_sudoku': sudoku_puzzle,
                    'sudoku_solution': sudoku_solution,
                    'children': children_thoughts,
                    }

        checker = MyChecker()

        for ii in range(1, 21):
            k = ii*5
            tot_chain = ToTChain(llm=llm,
                                checker=MyChecker(),
                                k=k,
                                c=children_thoughts,
                                verbose=False,
                                verbose_llm=False)
                                
            solution = tot_chain.run(problem_description=problem_description)
            if solution == sudoku_solution:
                break
        
        temp_dict['final_solution'] = solution
        temp_dict['k'] = k
        data_config[model][file] = temp_dict


InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?